# Looking at the historical US market returns

## Imports

In [53]:
import pandas as pd
import matplotlib.pyplot as plt

## Data

In [54]:
df = pd.read_excel('../data/shiller_sp500.xls', sheet_name='Data')
print(df)

                                             Unnamed: 0  \
0     Stock Market Data Used in "Irrational Exuberan...   
1                                    Robert J. Shiller    
2                                                   NaN   
3                                                   NaN   
4                                                   NaN   
...                                                 ...   
1863                                            2025.09   
1864                                             2025.1   
1865                                            2025.11   
1866                                            2025.12   
1867                                                NaN   

                      Unnamed: 1 Unnamed: 2 Unnamed: 3  \
0                            NaN        NaN        NaN   
1                            NaN        NaN        NaN   
2                            NaN        NaN        NaN   
3                            NaN        NaN        NaN   
4